## Bayesian AGN Decomposition Analysis for SDSS Spectra (BADASS)
### Single Spectrum

####  Remington O. Sexton$^{1,2}$, Sara M. Doan$^{1}$, Michael A. Reefe$^{1}$, William Matzko$^{1}$
$^{1}$George Mason University, $^{2}$United States Naval Observatory


In [1]:
import glob
import time
import natsort
from IPython.display import clear_output
# import multiprocess as mp
import os
import psutil
import pathlib
import natsort
# Import BADASS here
import badass as badass
import badass_utils as badass_utils
import badass_ncomp_options

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

### BADASS Options

In [2]:
options_file = "badass_ncomp_options.py"
if os.path.exists(options_file):
    print("\n Options file %s found.\n" % (options_file))


 Options file badass_ncomp_options.py found.



### Run BADASS on a single spectrum

The following is shows how to fit single SDSS spectra.

#### Directory Structure

In [3]:
nobj = 1 # Object in the spec_dir list
########################## Directory Structure #################################
spec_dir = 'examples/' # folder with spectra in it
# Get full list of spectrum folders; these will be the working directories
spec_loc = natsort.natsorted( glob.glob(spec_dir+'*') )[nobj]

################################################################################
print(len(spec_loc))
print(spec_loc)


15
examples/1-test


#### Choose Spectrum 

In [4]:
file = glob.glob(spec_loc+'/*.fits')[0] # Get name of FITS spectra file
print(file)

examples/1-test/spec-7748-58396-0782.fits


#### Run IRSA Dust Query
To correct for Galactic extinction.  This only needs to be done once so that the data is stored locally.

In [5]:
# badass_utils.fetch_IRSA_dust(spec_loc)

#### Run 

In [ ]:
import importlib
import numpy as np
importlib.reload(badass)
importlib.reload(badass_utils)
importlib.reload(badass_ncomp_options)


for i in range(5):
    np.random.seed()
    print("\n---------------------------------------------------------")
    print(" Begin Test %d of %d" % (i+1,10))
    print("---------------------------------------------------------")
    
    # Call the main function in BADASS
    badass.run_BADASS(pathlib.Path(file),
                      options_file = options_file,
                     )
        #


---------------------------------------------------------
 Begin Test 1 of 10
---------------------------------------------------------
Start process memory: 0.52212941                    
 Folder has not been created.  Creating MCMC_output folder...

 > Starting fit for 1-test


-----------------------------------------------------------
 file:                        spec-7748-58396-0782.fits     
 SDSS redshift:               0.58427                       
 fitting region:              (4400,5500) [A]               
 velocity scale:              69.11 [km/s/pixel]            
 Galactic E(B-V):             0.041                         
 Flux Normalization:          1.0e-17                       
-----------------------------------------------------------
	 - Adding parameter for unexplained noise to fit reduced Chi-squared.
	 - Fitting a SSP host-galaxy template.
  - Fitting additive legendre polynomial component.
	 - Fitting Simple AGN power-law continuum.
	 - Fitting broad and nar

basinhopping step 16: f -197.389 trial_f inf accepted 0  lowest_f -197.389
-197.38942405837196 15
at minimum inf accepted 0
 Fit Status: False
 Force threshold: 17.18
 Lowest RMSE: 12.75
 Current RMSE: 1250.26
 RMSE MAD: 0.22
 RMSE STD: 0.29
 RMSE Threshold: 13.33
 Accepted count: 1
 Basinhop count: 16


basinhopping step 17: f -197.389 trial_f 328.459 accepted 0  lowest_f -197.389
-197.38942405837196 16
at minimum 328.4585 accepted 0
 Fit Status: False
 Force threshold: 17.18
 Lowest RMSE: 12.75
 Current RMSE: 254.59
 RMSE MAD: 0.22
 RMSE STD: 0.28
 RMSE Threshold: 13.34
 Accepted count: 1
 Basinhop count: 17


basinhopping step 18: f -197.389 trial_f inf accepted 0  lowest_f -197.389
-197.38942405837196 17
at minimum inf accepted 0
 Fit Status: False
 Force threshold: 17.18
 Lowest RMSE: 12.75
 Current RMSE: 870.14
 RMSE MAD: 0.23
 RMSE STD: 0.31
 RMSE Threshold: 13.35
 Accepted count: 1
 Basinhop count: 18


basinhopping step 19: f -197.389 trial_f -14.1901 accepted 0  lowest_f -197

basinhopping step 40: f -202.809 trial_f 332.349 accepted 0  lowest_f -202.809
-202.80873149351797 15
at minimum 332.3491 accepted 0
 Fit Status: False
 Force threshold: 17.18
 Lowest RMSE: 12.33
 Current RMSE: 254.55
 RMSE MAD: 0.26
 RMSE STD: 0.36
 RMSE Threshold: 13.37
 Accepted count: 2
 Basinhop count: 16


basinhopping step 41: f -202.809 trial_f 315.476 accepted 0  lowest_f -202.809
-202.80873149351797 16
at minimum 315.4760 accepted 0
 Fit Status: False
 Force threshold: 17.18
 Lowest RMSE: 12.33
 Current RMSE: 238.61
 RMSE MAD: 0.22
 RMSE STD: 0.35
 RMSE Threshold: 13.31
 Accepted count: 2
 Basinhop count: 17


basinhopping step 42: f -202.809 trial_f 310.169 accepted 0  lowest_f -202.809
-202.80873149351797 17
at minimum 310.1689 accepted 0
 Fit Status: False
 Force threshold: 17.18
 Lowest RMSE: 12.33
 Current RMSE: 227.10
 RMSE MAD: 0.25
 RMSE STD: 0.35
 RMSE Threshold: 13.31
 Accepted count: 2
 Basinhop count: 18


basinhopping step 43: f -202.809 trial_f -133.086 accepted

basinhopping step 9: f -174.172 trial_f inf accepted 0  lowest_f -174.172
-174.17236610204938 8
at minimum inf accepted 0
 Fit Status: False
 Force threshold: 12.33
 Lowest RMSE: 13.90
 Current RMSE: 882.35
 RMSE MAD: 0.18
 RMSE STD: 0.29
 RMSE Threshold: 14.48
 Accepted count: 1
 Basinhop count: 9


basinhopping step 10: f -174.172 trial_f 339.822 accepted 0  lowest_f -174.172
-174.17236610204938 9
at minimum 339.8223 accepted 0
 Fit Status: False
 Force threshold: 12.33
 Lowest RMSE: 13.90
 Current RMSE: 266.40
 RMSE MAD: 0.13
 RMSE STD: 0.28
 RMSE Threshold: 14.42
 Accepted count: 1
 Basinhop count: 10


basinhopping step 11: f -174.172 trial_f 325.478 accepted 0  lowest_f -174.172
-174.17236610204938 10
at minimum 325.4779 accepted 0
 Fit Status: False
 Force threshold: 12.33
 Lowest RMSE: 13.90
 Current RMSE: 239.34
 RMSE MAD: 0.18
 RMSE STD: 0.30
 RMSE Threshold: 14.48
 Accepted count: 1
 Basinhop count: 11


basinhopping step 12: f -174.172 trial_f -120.293 accepted 0  lowest_f 

basinhopping step 32: f -174.172 trial_f 322.4 accepted 0  lowest_f -174.172
-174.17236610204938 31
at minimum 322.4002 accepted 0
 Fit Status: False
 Force threshold: 12.33
 Lowest RMSE: 13.90
 Current RMSE: 239.54
 RMSE MAD: 0.21
 RMSE STD: 0.26
 RMSE Threshold: 14.52
 Accepted count: 1
 Basinhop count: 32


basinhopping step 33: f -174.172 trial_f 187.718 accepted 0  lowest_f -174.172
-174.17236610204938 32
at minimum 187.7180 accepted 0
 Fit Status: False
 Force threshold: 12.33
 Lowest RMSE: 13.90
 Current RMSE: 102.35
 RMSE MAD: 0.17
 RMSE STD: 0.26
 RMSE Threshold: 14.48
 Accepted count: 1
 Basinhop count: 33


basinhopping step 34: f -174.172 trial_f inf accepted 0  lowest_f -174.172
-174.17236610204938 33
at minimum inf accepted 0
 Fit Status: False
 Force threshold: 12.33
 Lowest RMSE: 13.90
 Current RMSE: 859.12
 RMSE MAD: 0.18
 RMSE STD: 0.26
 RMSE Threshold: 14.49
 Accepted count: 1
 Basinhop count: 34


basinhopping step 35: f -174.172 trial_f 142.325 accepted 0  lowest_f

...

basinhopping step 52: f -165.343 trial_f inf accepted 0  lowest_f -165.343
-165.34289771367625 51
at minimum inf accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 1168.99
 RMSE MAD: 0.21
 RMSE STD: 0.29
 RMSE Threshold: 14.08
 Accepted count: 1
 Basinhop count: 52


basinhopping step 53: f -165.343 trial_f 102.882 accepted 0  lowest_f -165.343
-165.34289771367625 52
at minimum 102.8817 accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 73.20
 RMSE MAD: 0.20
 RMSE STD: 0.29
 RMSE Threshold: 14.07
 Accepted count: 1
 Basinhop count: 53


basinhopping step 54: f -165.343 trial_f 343.352 accepted 0  lowest_f -165.343
-165.34289771367625 53
at minimum 343.3515 accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 252.37
 RMSE MAD: 0.21
 RMSE STD: 0.29
 RMSE Threshold: 14.08
 Accepted count: 1
 Basinhop count: 54


basinhopping step 55: f -165.343 trial_f 334.839 accepted 0  lowest

basinhopping step 75: f -165.343 trial_f 352.487 accepted 0  lowest_f -165.343
-165.34289771367625 74
at minimum 352.4872 accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 266.40
 RMSE MAD: 0.22
 RMSE STD: 0.28
 RMSE Threshold: 14.08
 Accepted count: 1
 Basinhop count: 75


basinhopping step 76: f -165.343 trial_f inf accepted 0  lowest_f -165.343
-165.34289771367625 75
at minimum inf accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 1146.08
 RMSE MAD: 0.21
 RMSE STD: 0.28
 RMSE Threshold: 14.08
 Accepted count: 1
 Basinhop count: 76


basinhopping step 77: f -165.343 trial_f inf accepted 0  lowest_f -165.343
-165.34289771367625 76
at minimum inf accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 672.51
 RMSE MAD: 0.20
 RMSE STD: 0.28
 RMSE Threshold: 14.06
 Accepted count: 1
 Basinhop count: 77


basinhopping step 78: f -165.343 trial_f 367.135 accepted 0  lowest_f -165.

basinhopping step 98: f -165.343 trial_f inf accepted 0  lowest_f -165.343
-165.34289771367625 97
at minimum inf accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 1275.81
 RMSE MAD: 0.25
 RMSE STD: 0.29
 RMSE Threshold: 14.16
 Accepted count: 1
 Basinhop count: 98


basinhopping step 99: f -165.343 trial_f 343.029 accepted 0  lowest_f -165.343
-165.34289771367625 98
at minimum 343.0292 accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 253.24
 RMSE MAD: 0.25
 RMSE STD: 0.29
 RMSE Threshold: 14.16
 Accepted count: 1
 Basinhop count: 99


adaptive stepsize: acceptance rate 0.000000 target 0.500000 new stepsize 81 old stepsize 90
basinhopping step 100: f -165.343 trial_f inf accepted 0  lowest_f -165.343
-165.34289771367625 99
at minimum inf accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 526.99
 RMSE MAD: 0.25
 RMSE STD: 0.29
 RMSE Threshold: 14.16
 Accepted count: 1
 Bas

basinhopping step 121: f -165.343 trial_f 194.137 accepted 0  lowest_f -165.343
-165.34289771367625 120
at minimum 194.1374 accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 116.25
 RMSE MAD: 0.27
 RMSE STD: 0.30
 RMSE Threshold: 14.17
 Accepted count: 1
 Basinhop count: 121


basinhopping step 122: f -165.343 trial_f inf accepted 0  lowest_f -165.343
-165.34289771367625 121
at minimum inf accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 430.30
 RMSE MAD: 0.27
 RMSE STD: 0.30
 RMSE Threshold: 14.17
 Accepted count: 1
 Basinhop count: 122


basinhopping step 123: f -165.343 trial_f 343.378 accepted 0  lowest_f -165.343
-165.34289771367625 122
at minimum 343.3776 accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 252.48
 RMSE MAD: 0.27
 RMSE STD: 0.30
 RMSE Threshold: 14.17
 Accepted count: 1
 Basinhop count: 123


basinhopping step 124: f -165.343 trial_f 195.097 accepted

basinhopping step 144: f -165.343 trial_f inf accepted 0  lowest_f -165.343
-165.34289771367625 143
at minimum inf accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 1154.25
 RMSE MAD: 0.27
 RMSE STD: 0.30
 RMSE Threshold: 14.19
 Accepted count: 1
 Basinhop count: 144


basinhopping step 145: f -165.343 trial_f 220.474 accepted 0  lowest_f -165.343
-165.34289771367625 144
at minimum 220.4736 accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 123.67
 RMSE MAD: 0.27
 RMSE STD: 0.30
 RMSE Threshold: 14.19
 Accepted count: 1
 Basinhop count: 145


basinhopping step 146: f -165.343 trial_f inf accepted 0  lowest_f -165.343
-165.34289771367625 145
at minimum inf accepted 0
 Fit Status: False
 Force threshold: 11.65
 Lowest RMSE: 13.47
 Current RMSE: 849.64
 RMSE MAD: 0.27
 RMSE STD: 0.30
 RMSE Threshold: 14.19
 Accepted count: 1
 Basinhop count: 146


basinhopping step 147: f -165.343 trial_f -129.039 accepted 0  low